In [ ]:
import sys, os, glob

import BreakJunctionUtilities as bju

import numpy as np
import pandas as pd

import pickle

# Load Data

In [ ]:
directory = 'INSERT DIRECTORY' # Directory to a folder containing subdirectories hosting BJ data mat files
folders=[]
files = os.listdir(directory)
for file in files:
    if os.path.isdir(os.path.join(directory, file)):
        folders.append(file)
folders

In [ ]:
# Load BJ data files created by BJSaveData.m
raw_files = []
file_pattern = '*{}.mat' # File pattern for files to load
# file_pattern = '*BJ*.mat'
# file_pattern = '*.mat'
for folder in folders:
    full_path = os.path.join(directory, folder, file_pattern.format(folder))
    path = glob.glob(full_path)[0]
    raw = bju.loadmat(path)
    raw_files.append(raw)   

# Format BJ Data

In [ ]:
# Plateau Marks Constants
start_mark = -0.1
end_mark = -5.0
sampling_points = 10

In [ ]:
def calc_plat_len(x, y, start, end, samp):
    '''
    Calculates the x-length of a BJ plateau using a start and end y values.
    '''
    z1 = None
    z2 = None
    for i in range(len(y) - samp):
        vals = y[i : i+samp]
        avg = vals.mean()
        if (avg < start) & (z1 == None):
            z1 = x[int(i + (samp / 2))]
        if (avg < end) & (z1 != None) & (z2 == None):
            z2 = x[int(i + (samp / 2))]
            break
    if (z2 == None) or (z1 == None):
        return None
    else:
        length = z2 - z1
        return length
    
    
        
def format_data(raw_files):
    '''
    Formats BJ data into a DataFrame.
    '''
    columns = ['logG', 'Z', 'PlateauLen', 'speed', 'setpoint', 'bias', 'trial']
    df = pd.DataFrame(columns=columns)
    
    for i in range(len(raw_files)):
        trial = raw_files[i]
        trial_num = folders[i]
        trial_traces = trial['importedfiles']
        for trace in trial_traces:
            logG = trace['logG']
            Z = trace['Z']
            plat = calc_plat_len(Z, logG, start_mark, end_mark, sampling_points)
            speed = trace['header']['speed']
            setpoint = trace['header']['setpoint']
            bias = trace['header']['bias']
            
            values = [[logG], [Z], plat, speed, setpoint, bias, trial_num]
            entry = pd.DataFrame(dict(zip(columns, values)))
            df = pd.concat((df, entry))
    return df

data_df = format_data(raw_files)
data_df

# Save Formatted BJ Data

In [ ]:
data_df.to_pickle(os.path.join(directory, "INSERT FILENAME"))